In [65]:
%%bash 
rm -r Data
mkdir Data
cd Data
kg download -c "poker-rule-induction"

downloading https://www.kaggle.com/c/poker-rule-induction/download/sampleSubmission.csv.zip

downloading https://www.kaggle.com/c/poker-rule-induction/download/train.csv.zip

downloading https://www.kaggle.com/c/poker-rule-induction/download/test.csv.zip



sampleSubmission.csv.zip 100% |######################| Time: 0:00:09 124.0 KiB/s
train.csv.zip 100% |#################################| Time: 0:00:02  54.6 KiB/s
test.csv.zip 100% |##################################| Time: 0:00:57 119.6 KiB/s


In [66]:
%%bash 
cd Data
unzip test.csv.zip
unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [15]:
%matplotlib inline
import pandas as pd
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.animation import FuncAnimation
from tempfile import NamedTemporaryFile
from IPython.display import HTML
import seaborn as sns
import numpy as np
import six
import tensorflow as tf

plt.style.use('seaborn-talk')
sns.set_context("talk", font_scale=1.4)

### Preprocess Data

In [138]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
lb = LabelBinarizer()
scaler = MinMaxScaler()
def pre_process(data):
    y = data["hand"].values
    x = data.drop("hand",axis=1).values
    x = scaler.fit_transform(x)
    lb.fit(y)
    y = lb.transform(y)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.03, shuffle=True)
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return (x_train, x_test, y_train, y_test,lb ,scaler)

In [139]:
# ?plt.scatter

In [140]:
train_data = pd.read_csv("Data/train.csv")
(x_train, x_test, y_train, y_test,lb,scaler) = pre_process(train_data)
del train_data

   S1  C1  S2  C2  S3  C3  S4  C4  S5  C5  hand
0   4   9   2   1   2   2   4   7   2   8     0
1   1   4   3   6   1  12   3  11   2   7     0
2   1  11   4   1   3   7   4  11   2   1     2
3   2   9   2   4   3   6   1   9   4   9     3
4   1   8   2   4   2  11   2   2   2   1     0


AttributeError: 'list' object has no attribute 'size'

In [125]:
x_train.shape

(24259, 10)

In [19]:
x_test.shape

(751, 10)

In [20]:
y_train.shape

(24259, 10)

In [118]:
y_test.shape

(751, 10)

In [74]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
import keras.backend as K
from keras import losses
import keras
import h5py

In [84]:
model = Sequential()
#input layer
model.add(Dense(100, input_shape=x_train[0].shape))
model.add(Activation("relu"))

# hidden layers
model.add(Dense(200))
model.add(Activation("relu"))

model.add(Dense(400))
model.add(Activation("relu"))
    
model.add(Dense(2, activation="relu"))
    
# output layer
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 100)               1100      
_________________________________________________________________
activation_22 (Activation)   (None, 100)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 200)               20200     
_________________________________________________________________
activation_23 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 400)               80400     
_________________________________________________________________
activation_24 (Activation)   (None, 400)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 802       
__________

In [95]:
model.save("hello.h5")

In [88]:
for i in xrange(1,3):
    model.fit(x_train, y_train, epochs=15, batch_size=20)
    score = model.evaluate(x_test, y_test)
    print(score)

Epoch 1/15
24259/24259 [==============================] - 6s - loss: 0.0465 - acc: 0.9902     
Epoch 2/15
24259/24259 [==============================] - 6s - loss: 0.0513 - acc: 0.9890     
Epoch 3/15
24259/24259 [==============================] - 6s - loss: 0.0482 - acc: 0.9895     
Epoch 4/15
24259/24259 [==============================] - 6s - loss: 0.0460 - acc: 0.9901     
Epoch 5/15
24259/24259 [==============================] - 6s - loss: 0.0459 - acc: 0.9901     
Epoch 6/15
24259/24259 [==============================] - 6s - loss: 0.0640 - acc: 0.9869     
Epoch 7/15
24259/24259 [==============================] - 6s - loss: 0.0474 - acc: 0.9900     
Epoch 8/15
24259/24259 [==============================] - 6s - loss: 0.0545 - acc: 0.9880     
Epoch 9/15
24259/24259 [==============================] - 6s - loss: 0.0469 - acc: 0.9895     
Epoch 10/15
24259/24259 [==============================] - 6s - loss: 0.0460 - acc: 0.9900     
Epoch 11/15
24259/24259 [========================

In [103]:
def pre_process_predictdata(data):
    data = data.drop(["id"],axis=1)
    data = scaler.fit_transform(data.values)
    return data

In [104]:
test_data = pd.read_csv("Data/test.csv")

In [107]:
to_predict_data = pre_process_predictdata(test_data)

In [119]:
prediction = model.predict(to_predict_data,batch_size=128)

In [109]:
prediction[0]

array([  9.96958613e-01,   1.04208884e-04,   6.06416467e-15,
         9.33758199e-33,   1.29569834e-03,   1.64157234e-03,
         1.87041255e-35,   1.72801093e-10,   3.81799214e-09,
         1.27606103e-08], dtype=float32)

In [124]:
len(prediction)

1000000